<a href="https://colab.research.google.com/github/sahandv/science_science/blob/master/Elmo_contextual_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ELMo

Note that you will need to use the non-GPU accelerated run-time on this notebook due to the large memory useage of the ELMo model.

## Imports:

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing

!python -m spacy download en_core_web_md #you will need to install this on first load, then restart the kernel!? \(oO)/
import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_md')
from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


## Get the data 

The below downloads a Pandas Dataframe which is publically hosted on Google Drive (this should therefore work for anyone)

In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


file_id = '1M_XljfV5t_nGjvhyfTPO9n2nfOweMwYx'
destination = 'temp'
download_file_from_google_drive(file_id, destination)

combined = pd.read_pickle('temp')

combined.head()

In [13]:
f = open("temp", "r")
print(f.read())

<HTML>
<HEAD>
<TITLE>Not Found</TITLE>
</HEAD>
<BODY BGCOLOR="#FFFFFF" TEXT="#000000">
<H1>Not Found</H1>
<H2>Error 404</H2>
</BODY>
</HTML>



## Create sentence embeddings

In [ ]:
url = "https://tfhub.dev/google/elmo/3"
embed = hub.Module(url)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Downloaded https://tfhub.dev/google/elmo/2, Total size: 357.40MB
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/2'.


In [ ]:
combined.loc[combined.Company.str.contains("Asos")]

,Company,URL,Industry,HQ,Also Covers Companies,UK Modern Slavery Act,California Transparency in Supply Chains Act,Period Covered,text,pdf,error,FT_tfidf
494,Asos plc,https://www.asosplc.com/~/media/Files/A/Asos-V...,Internet & Direct Marketing Retail,United Kingdom,NaN,True,False,2016-2018,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,0,"[-0.8573993510860287, 0.11926992131730585, -0...."
495,Asos plc,https://www.asosplc.com/~/media/Files/A/ASOS/r...,Internet & Direct Marketing Retail,United Kingdom,NaN,True,False,2015-2016,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,0,"[-0.6513810935113411, 0.04600498146602333, 0.0..."


In [ ]:
text = combined.iloc[494].text
import re

text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
text = ' '.join(text.split())
doc = nlp(text)

sentences = []
for i in doc.sents:
  if len(i)>1:
    sentences.append(i.string.strip())
    
len(sentences)

389

In [ ]:
sentences[0:10]

['modern slavery statement september 2016 - march 2018',
 'facts & figures - 31 august 2017 slavery, servitude, forced labour, bonded labour, and human trafficking are issues of increasing global concern, affecting all sectors, regions and economies.',
 'modern slavery is fundamentally unacceptable within our business and supply chain, and combatting it is an important element of our overall approach to business and human rights.',
 'asos is committed to respecting, protecting and championing the human rights of all those who come into contact with our operations, including employees, supply chain workers, customers and local communities.',
 'we accept our responsibility to support transparency; to find and resolve problems, to regularly review our business practices, and to collaborate with others to protect the rights of workers, particularly those who are most vulnerable to abuses such as modern slavery.',
 'this statement has been published in accordance with the modern slavery act

In [ ]:
embeddings = embed(
    sentences,
    signature="default",
    as_dict=True)["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
%%time
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  x = sess.run(embeddings)

CPU times: user 3min 22s, sys: 16.7 s, total: 3min 39s
Wall time: 1min 53s


## Visualize the sentences using PCA and TSNE

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
y = pca.fit_transform(x)

from sklearn.manifold import TSNE

y = TSNE(n_components=2).fit_transform(y)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


data = [
    go.Scatter(
        x=[i[0] for i in y],
        y=[i[1] for i in y],
        mode='markers',
        text=[i for i in sentences],
    marker=dict(
        size=16,
        color = [len(i) for i in sentences], #set color equal to a variable
        opacity= 0.8,
        colorscale='Viridis',
        showscale=False
    )
    )
]
layout = go.Layout()
layout = dict(
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
fig = go.Figure(data=data, layout=layout)
file = plot(fig, filename='Sentence encode.html')

from google.colab import files
files.download('Sentence encode.html') 

## Create a semantic search engine:

In [ ]:
#@title Sementic search
#@markdown Enter a set of words to find matching sentences. 'results_returned' can beused to modify the number of matching sentences retured. To view the code behind this cell, use the menu in the top right to unhide...
search_string = "code of ethics" #@param {type:"string"}
results_returned = "3" #@param [1, 2, 3]

from sklearn.metrics.pairwise import cosine_similarity


embeddings2 = embed(
    [search_string],
    signature="default",
    as_dict=True)["default"]

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  search_vect = sess.run(embeddings2)
  

cosine_similarities = pd.Series(cosine_similarity(search_vect, x).flatten())
output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
  output +='<p style="font-family:verdana; font-size:110%;"> '
  for i in sentences[i].split():
    if i.lower() in search_string:
      output += " <b>"+str(i)+"</b>"
    else:
      output += " "+str(i)
  output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))
#   print(sentences[i])
#   print('\n')
